In [1]:
import os

workspace_path="/home/hpclqz/share/project/04_TPBench/congestion_test/workspace" 
time_stamp = "-20250208_210330"   
time_stamp = ''        
time_stamp = "-20250212_125541"   

interleaved_path = os.path.join(workspace_path, f"logs{time_stamp}-interleaved")
single_path = os.path.join(workspace_path, f"logs{time_stamp}-single")

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import plot


class EnvironmentPara:
    def __init__(self, task_type:str, node_num:int, node_list:list[str], start_time, end_time):
        self.node_num:int = node_num
        self.node_list = node_list
        self.time_start = start_time
        self.time_end = end_time
        self.task_type = task_type
        self.date_time = self.get_date_time(start_time, end_time)

    def __str__(self):
        return f"{self.name}: {self.workload}"

    def get_date_time(self, start_time, end_time):
        # start time: 20241227-230036
        # end time: 20241227-230048
        s_date = start_time.split('-')[0]
        s_time = start_time.split('-')[1]
        s_hour = int(s_time[:2])
        s_min = int(s_time[2:4])
        s_sec = int(s_time[4:])

        e_date = end_time.split('-')[0]
        e_time = end_time.split('-')[1]
        e_hour = int(e_time[:2])
        e_min = int(e_time[2:4])
        e_sec = int(e_time[4:])

        date = f"{e_date[0:4]}-{e_date[4:6]}-{e_date[6:8]}"
        time = ""
        if s_date == e_date:
            if e_hour == s_hour:
                if e_min == s_min:
                    if s_sec <= 60 - e_sec:
                        time = f"{s_hour:02}:{s_min:02}:00"
                    else:
                        time = f"{s_hour:02}:{s_min:02}:00"
                else:
                    time = f"{s_hour:02}:{s_min:02}:00"
            else:
                time = f"{e_hour:02}:{e_min:02}:00"
        else:
            time = f"{e_hour:02}:{e_min:02}:00"


        return f"{date} {time}"

class ProgramPara:
    def __init__(self, lines:list[str]):
       
        self.vars = 0
        self.iteration = 0
        self.Time = 0
        self.KBytesXchng_per_rank_max = 0.0
        self.MB_per_sec_per_rank = 0.0

        self.Msgs_per_sec = 0.0
        self.MB_per_sec = 0.0

        for i,line in enumerate(lines):
            if  "Message Size" in line:
                if line.split()[-1].isdigit():
                    self.vars = int(line.split()[-1])
                elif line.split()[-2].isdigit():
                    self.vars = int(line.split()[-2])
            elif "Repeats" in line:
                self.iteration = int(line.split()[-1])
            elif line.startswith("#   MsgSize        Time             KMsgs             MB           KMsg/S           MB/S"):
                # pingpang
                line = lines[i+1]
                self.Time = float(line.split()[1])
                self.KMsgs = float(line.split()[2])
                self.MB = float(line.split()[3])
                self.KMsg_per_sec = float(line.split()[4])
                self.MB_per_sec = float(line.split()[5])

    def get_speed(self):
        if self.MB_per_sec_per_rank:
            return self.MB_per_sec_per_rank
        elif self.MB_per_sec:
            return self.MB_per_sec

    def get_time(self):
        return self.Time


class Task:
    def __init__(self, env_para: EnvironmentPara, program_para: ProgramPara):
        self.kernel_name = 'pingpong'
        self.env_para = env_para
        self.program_para = program_para
        self.date_time = self.env_para.date_time
        self.task_type = self.env_para.task_type
        self.time = self.program_para.get_time()
        self.speed = self.program_para.get_speed()


                    
                            
                            

class Tasks:
    speed_map = {"incast":"MB_per_sec", "pingpong":"MB_per_sec"}
    headers = ['variables', 'node_num', 'node_list', 'date_time', 'task_type', 'total_time', 'MB/s']
    types = [int, int, str, str, str, float, float]

    def __init__(self, log_path:str, task_type:str, update_tasks=True):
        self.tasks = [] 
        self.task_type = task_type
        file_name_list = os.listdir(log_path)
        self.data = pd.DataFrame(columns=Tasks.headers)

        if update_tasks or os.path.exists(f'data{time_stamp}-{task_type}.csv') == False:
            self.update_tasks_datas(file_name_list, log_path)
        else:
            self.data = pd.read_csv(f'data{time_stamp}-{task_type}.csv', dtype=dict(zip(Tasks.headers, Tasks.types)))
    
    def update_tasks_datas(self, file_name_list, log_path):
        data_row = []
        for file_name in file_name_list:
            if file_name.endswith('.txt'):
                file_path = os.path.join(log_path, file_name)
                with open(file_path, 'r') as f:
                    node_num = 0
                    node_infos:str = ''
                    node_list = []
                    task_type = ''
                    
                    start_time, end_time = '', ''
                    start_line, end_line = 0, 0
                    lines = f.readlines()
                    for i,line in enumerate(lines):
                        if line.startswith("Node Number:"):
                            node_num = int(line.split()[-1])
                        elif line.startswith("Node List:"):
                            node_list = line.split()[-1]
                        elif line.startswith("Task Type:"):
                            task_type = line.split()[-1]
                        elif line.startswith("Start time:"):
                            tokens = line.split()
                            start_time = tokens[-1]
                            start_line = i + 1
                        elif line.startswith("End time:"):
                            tokens = line.split()
                            end_time = tokens[-1]
                            end_line = i
                            env_para = EnvironmentPara(task_type, node_num, node_list, start_time, end_time)
                            program_para = ProgramPara(lines[start_line:end_line])
                            task = Task(env_para, program_para)
                            if program_para.vars > 0 and task.time > 0 and task.speed > 0:
                                data_row.append([program_para.vars, node_num, node_list, env_para.date_time, 
                                                task_type, task.time, task.speed])
                                
                                self.tasks.append(task)
                            start_time, end_time = '', ''
                            start_line, end_line = 0, 0

        self.data = pd.DataFrame(data_row, columns=Tasks.headers)
        self.data.sort_values(['node_list', 'date_time', 'task_type', 'variables'], inplace=True)
        # print(self.data)
        self.data.to_csv(f'data{time_stamp}-{self.task_type}.csv', index=False)
        
    def plot_histogram(self, save_path:str):
        variables_list = self.data['variables'].unique()
        for variables in variables_list:
            data_var = self.data[self.data['variables'] == variables]
            title_base = f"Histogram of {self.task_type} with {variables} Bytes"
            node_lists = data_var['node_list'].unique()
            for node_list in node_lists:
                data_node = data_var[data_var['node_list'] == node_list]
                title = f"{title_base}\n{node_list}"
                file_name = f"Histogram-{variables}-{node_list}-{self.task_type}"
                plot.plot_histogram(data=data_node, title=title, file_name=file_name, save_path=save_path, x='total_time')
                plot.plot_histogram(data=data_node, title=title, file_name=file_name, save_path=save_path, x='MB/s')

    def compare_histogram(self, task2:Task, save_path:str, title:str):
        variables_list_1 = self.data['variables'].unique()
        variables_list_2 = task2.data['variables'].unique()
        variable_list = list(set(variables_list_1) & set(variables_list_2))
        for variable in variable_list:
            data_var_1 = self.data[self.data['variables'] == variable]
            data_var_2 = task2.data[task2.data['variables'] == variable]
            title_base = f"Histogram of {self.task_type} with {variable} Bytes"
            node_lists_1 = data_var_1['node_list'].unique()
            node_lists_2 = data_var_2['node_list'].unique()
            node_list = list(set(node_lists_1) & set(node_lists_2))
            for node in node_list:
                data_node_1 = data_var_1[data_var_1['node_list'] == node]
                data_node_2 = data_var_2[data_var_2['node_list'] == node]
                title = f"{title_base}\n{node}"
                file_name = f"{self.task_type}-{node}"
                plot.plot_histogram(data1=data_node_1, data2=data_node_2, title=title, file_name=file_name, save_path=save_path, x='total_time')
                plot.plot_histogram(data1=data_node_1, data2=data_node_2, title=title, file_name=file_name, save_path=save_path, x='MB/s')
                



tasks_inter = Tasks(interleaved_path, task_type="interleaved", update_tasks=False)
tasks_single = Tasks(single_path, task_type="single", update_tasks=False)


In [3]:
import os

workspace_path="/home/hpclqz/share/project/04_TPBench/congestion_test/workspace"                      
figure_path = os.path.join(workspace_path,f"figures{time_stamp}-origin")
os.makedirs(figure_path, exist_ok=True)
# tasks_inter.plot_histogram(figure_path)
# tasks_single.plot_histogram(figure_path)
# tasks_inter.compare_histogram(tasks_single, figure_path)

In [ ]:
import plot

class Tasks_Statistic():
    hostfile_pair_list:list[tuple] = [("hostfile1", "hostfile2"), ("hostfile3", "hostfile4"), ("hostfile5", "hostfile6")]
    nodes_pair_list:list[tuple] = [("pi1,pi2", "pi3,pi4"), ("pi1,pi3", "pi2,pi4"), ("pi1,pi4", "pi2,pi3"), ("pi2,pi3", "pi1,pi4"), ("pi2,pi4", "pi1,pi3"), ("pi3,pi4", "pi1,pi2")]
    headers = ['nodes', 'variables', 'task_type', 'perf_type', 'average', 'std', 'max', 'min', '1%', '5%', '1%_avg', '5%_avg']
    def __init__(self, tasks_inter:Tasks, tasks_single:Tasks, update_data=True):
        self.data:pd.DataFrame = None
        if update_data or os.path.exists(f'data{time_stamp}-statistic.csv') == False:
            self.nodes_perf_statistic(tasks_inter)
            self.nodes_perf_statistic(tasks_single)
            self.data.sort_values(['perf_type', 'nodes', 'variables', 'task_type'], inplace=True)
            self.data.to_csv(f'data{time_stamp}-statistic.csv', index=False)
        else:
            self.data = pd.read_csv(f'data{time_stamp}-statistic.csv')

    def nodes_perf_statistic(self, tasks:Tasks):
        '''
        
        '''
        data_row = []
        node_list = tasks.data['node_list'].unique()
        for node in node_list:
            data_node = tasks.data[tasks.data['node_list'] == node]
            variables_list = data_node['variables'].unique()
            for variables in variables_list:
                data_var = data_node[data_node['variables'] == variables]
                task_type_list = data_var['task_type'].unique()
                for task_type in task_type_list:
                    data_type = data_var[data_var['task_type'] == task_type]
                    for perf_type in ['total_time', 'MB/s']:
                        average = data_type[perf_type].mean()
                        std = data_type[perf_type].std()
                        max = data_type[perf_type].max()
                        min = data_type[perf_type].min()
                        if 'time' in perf_type:
                            one_percent = data_type[perf_type].quantile(0.99)
                            five_percent = data_type[perf_type].quantile(0.95)
                            average_one_percent = data_type[data_type[perf_type] > one_percent][perf_type].mean()
                            average_five_percent = data_type[data_type[perf_type] > five_percent][perf_type].mean()
                        else:
                            one_percent = data_type[perf_type].quantile(0.01)
                            five_percent = data_type[perf_type].quantile(0.05)
                            average_one_percent = data_type[data_type[perf_type] < one_percent][perf_type].mean()
                            average_five_percent = data_type[data_type[perf_type] < five_percent][perf_type].mean()
                        data_row.append([node, variables, task_type, perf_type, average, std, max, min, one_percent, five_percent, average_one_percent, average_five_percent])
                    
        if self.data is None:
            self.data = pd.DataFrame(data_row, columns=Tasks_Statistic.headers)
        else:
            self.data = pd.concat([self.data, pd.DataFrame(data_row, columns=Tasks_Statistic.headers)])

    def plot_statistic(self, save_path:str):
        '''
        绘制柱状图，每张图包含两对节点，柱状图的每个横轴是一个统计值：average,std,max,min,1%,5%,1%_avg,5%_avg，
        一个横轴坐标包含两个柱，其中一个柱是interleaved的，另一个柱是single的（归一化展示，single柱值为1）
        '''
        stats = ['average','std', '1%_avg','5%_avg']
        # For each statistic, create one figure that compares interleaved vs. single for a set of node pairs.
        for vars in self.data['variables'].unique():
            data_vars = self.data[self.data['variables'] == vars]
            for stat in stats:
                for perf_type in ['total_time', 'MB/s']:
                    data_pt = data_vars[data_vars['perf_type'] == perf_type]
                    title = f"Comparison of {stat} for {vars} Bytes in {perf_type}"
                    file_name = f"statistic-{vars}-{stat}-{perf_type.replace('/', 'per')}"
                    # data:pd.DataFrame, stat:str, title_format:str, save_path:str, file_name_format:str
                    plot.plot_bar_with_discrete_x(data=data_pt, x='nodes', stat=stat, title=title, file_name=file_name, save_path=save_path)
                    
tasks_statistic = Tasks_Statistic(tasks_inter, tasks_single)
figure_path = os.path.join(workspace_path, f"figures{time_stamp}-statistic")
os.makedirs(figure_path, exist_ok=True)
tasks_statistic.plot_statistic(figure_path)



In [ ]:
class Task_Paired():
    '''
    
    '''
    def __init__(self):
        self.kernel_name:str = 'pingpong'
        self.date_time:str = ''
        self.variables:int = 0
        self.node_pair1:str = ''
        self.node_pair2:str = ''
        self.nodepair1_inter:Task = None
        self.nodepair2_inter:Task = None
        self.nodepair1_single:Task = None
        self.nodepair2_single:Task = None
        self.speed_ave_inter = 0.0          # Average speed of interleaved node pair
        self.speed_ave_single = 0.0         # Average speed of single node pair
        self.time_ave_inter = 0.0
        self.time_ave_single = 0.0

    
class Task_Paired_List():
    hostfile_pair_list:list[tuple] = [("hostfile1", "hostfile2"), ("hostfile3", "hostfile4"), ("hostfile5", "hostfile6")]
    nodes_pair_list:list[tuple] = [("pi1,pi2", "pi3,pi4"), ("pi1,pi3", "pi2,pi4"), ("pi1,pi4", "pi2,pi3")]
    headers=['variables', 'date_time', 'node_pair1', 'node_pair2', 'time1_inter', 'time1_single', 'speed1_inter', 'speed1_single', 'time2_inter', 'time2_single', 'speed2_inter', 'speed2_single', 'time_ave_inter', 'time_ave_single', 'speed_ave_inter', 'speed_ave_single']
    def __init__(self, log_path:str, update_tasks=True):
        self.task_paired_list = []
        self.data:pd.DataFrame = None
        if update_tasks or os.path.exists(f'data{time_stamp}-paired.csv') == False:
            self.update_tasks_datas(log_path)
            self.data.sort_values(['variables', 'date_time', 'node_pair1', 'node_pair2'], inplace=True)
            self.data.to_csv(f'data{time_stamp}-paired.csv', index=False)
        else:
            self.data = pd.read_csv(f'data{time_stamp}-paired.csv')

    def update_tasks_datas(self, log_path):
        log_interleaved_path = interleaved_path 
        log_single_path = single_path
        file_inter_list = os.listdir(log_interleaved_path)
        file_single_list = os.listdir(log_single_path)
        time_list_dict:dict[str, list] = {}
        for file_name in file_inter_list:
            if file_name.endswith('.txt'):
                hostfile= file_name.split('-')[1]
                date=file_name.split('-')[2]
                time=file_name.split('-')[3].split('.')[0]
                data_time=f"{date}-{time}"
                if hostfile not in time_list_dict:
                    time_list_dict[hostfile] = []
                if data_time not in time_list_dict[hostfile]:
                    time_list_dict[hostfile].append(data_time)
        for i, hostfile_pair in enumerate(Task_Paired_List.hostfile_pair_list):
            nodes_pair = Task_Paired_List.nodes_pair_list[i]
            time_list = time_list_dict[hostfile_pair[0]]
            
            data_row = []
            for time in time_list:
                file1_inter_path=os.path.join(log_interleaved_path,f"log-{hostfile_pair[0]}-{time}.txt")
                file2_inter_path=os.path.join(log_interleaved_path,f"log-{hostfile_pair[1]}-{time}.txt")
                file1_single_path=os.path.join(log_single_path,f"log-{hostfile_pair[0]}-{time}.txt")
                file2_single_path=os.path.join(log_single_path,f"log-{hostfile_pair[1]}-{time}.txt")
                if os.path.exists(file1_inter_path) == False or os.path.exists(file2_inter_path) == False or os.path.exists(file1_single_path) == False or os.path.exists(file2_single_path) == False:
                    print(f"Warning: Does {file1_inter_path} exit? {os.path.exists(file1_inter_path)}")
                    print(f"Warning: Does {file2_inter_path} exit? {os.path.exists(file2_inter_path)}")
                    print(f"Warning: Does {file1_single_path} exit? {os.path.exists(file1_single_path)}")
                    print(f"Warning: Does {file2_single_path} exit? {os.path.exists(file2_single_path)}")
                    continue
                file_list = [file1_inter_path, file2_inter_path, file1_single_path, file2_single_path]
                file_task_dict:dict[str, list[Task]] = {}
                for file in file_list:
                    with open(file, 'r') as f:
                        node_num = 0
                        node_infos:str = ''
                        node_list = []
                        task_type = ''
                        start_time, end_time = '', ''
                        start_line, end_line = 0, 0
                        lines = f.readlines()
                        for i,line in enumerate(lines):
                            if line.startswith("Node Number:"):
                                node_num = int(line.split()[-1])
                            elif line.startswith("Node List:"):
                                node_list = line.split()[-1]
                            elif line.startswith("Task Type:"):
                                task_type = line.split()[-1]
                            elif line.startswith("Start time:"):
                                tokens = line.split()
                                start_time = tokens[-1]
                                start_line = i + 1
                            elif line.startswith("End time:"):
                                tokens = line.split()
                                end_time = tokens[-1]
                                end_line = i
                                env_para = EnvironmentPara(task_type, node_num, node_list, start_time, end_time)
                                program_para = ProgramPara(lines[start_line:end_line])
                                task = Task(env_para, program_para)
                                if program_para.vars > 0 and task.time > 0 and task.speed > 0:
                                    if file not in file_task_dict:
                                        file_task_dict[file] = []
                                    file_task_dict[file].append(task)
                                start_time, end_time = '', ''
                                start_line, end_line = 0, 0
                if len(file_task_dict) != 4:
                    print(f"Warning: file_task_dict is not 4")
                    print(f"Warning: {file1_inter_path} {file2_inter_path} {file1_single_path} {file2_single_path}")
                    continue
                if len(file_task_dict[file1_inter_path]) == len(file_task_dict[file2_inter_path]) and len(file_task_dict[file1_inter_path]) == len(file_task_dict[file1_single_path]) and len(file_task_dict[file1_inter_path]) == len(file_task_dict[file2_single_path]):
                    for i in range(len(file_task_dict[file1_inter_path])):
                        task_paired = Task_Paired()
                        task_paired.variables = file_task_dict[file1_inter_path][i].program_para.vars
                        task_paired.node_pair1 = nodes_pair[0]
                        task_paired.node_pair2 = nodes_pair[1]
                        task_paired.date_time = file_task_dict[file1_inter_path][i].date_time
                        task_paired.nodepair1_inter = file_task_dict[file1_inter_path][i]
                        task_paired.nodepair2_inter = file_task_dict[file2_inter_path][i]
                        task_paired.nodepair1_single = file_task_dict[file1_single_path][i]
                        task_paired.nodepair2_single = file_task_dict[file2_single_path][i]
                        task_paired.speed_ave_inter = (task_paired.nodepair1_inter.speed + task_paired.nodepair2_inter.speed) / 2
                        task_paired.speed_ave_single = (task_paired.nodepair1_single.speed + task_paired.nodepair2_single.speed) / 2
                        task_paired.time_ave_inter = (task_paired.nodepair1_inter.time + task_paired.nodepair2_inter.time) / 2
                        task_paired.time_ave_single = (task_paired.nodepair1_single.time + task_paired.nodepair2_single.time) / 2
                        self.task_paired_list.append(task_paired)
                        data_row.append([task_paired.variables, task_paired.date_time, task_paired.node_pair1, task_paired.node_pair2, task_paired.nodepair1_inter.time, task_paired.nodepair1_single.time, task_paired.nodepair1_inter.speed, task_paired.nodepair1_single.speed, task_paired.nodepair2_inter.time, task_paired.nodepair2_single.time, task_paired.nodepair2_inter.speed, task_paired.nodepair2_single.speed, task_paired.time_ave_inter, task_paired.time_ave_single, task_paired.speed_ave_inter, task_paired.speed_ave_single])
                else:
                    print(f"Warning: file_task_dict is not equal")
                    print(f"Warning: {file1_inter_path} {file2_inter_path} {file1_single_path} {file2_single_path}")
                    print(f"len(file_task_dict[file1_inter_path]) = {len(file_task_dict[file1_inter_path])}")
                    print(f"len(file_task_dict[file2_inter_path]) = {len(file_task_dict[file2_inter_path])}")
                    print(f"len(file_task_dict[file1_single_path]) = {len(file_task_dict[file1_single_path])}")
                    print(f"len(file_task_dict[file2_single_path]) = {len(file_task_dict[file2_single_path])}")
            if self.data is None:
                self.data = pd.DataFrame(data_row, columns=Task_Paired_List.headers)
            else:
                self.data = pd.concat([self.data, pd.DataFrame(data_row, columns=Task_Paired_List.headers)])

    def plot_histogram(self, save_path:str):
        variables_list = self.data['variables'].unique()
        for variables in variables_list:
            data_var = self.data[self.data['variables'] == variables]
            for node_pair in Task_Paired_List.nodes_pair_list:
                data_node = data_var[(data_var['node_pair1'] == node_pair[0]) & (data_var['node_pair2'] == node_pair[1])]
                if data_node.shape[0] == 0:
                    continue
                title_base = f"Histogram with {variables} Bytes"
                title = f"{title_base}\n{node_pair[0]} & {node_pair[1]}"
                file_name = f"Histogram-{variables}-{node_pair[0]}_{node_pair[1]}"
                plot.plot_histogram(data=data_node, title=title, file_name=file_name, save_path=save_path, x='time_ave_inter')
                plot.plot_histogram(data=data_node, title=title, file_name=file_name, save_path=save_path, x='time_ave_single')
                plot.plot_histogram(data=data_node, title=title, file_name=file_name, save_path=save_path, x='speed_ave_inter')
                plot.plot_histogram(data=data_node, title=title, file_name=file_name, save_path=save_path, x='speed_ave_single')
            
figure_path = os.path.join(workspace_path, f"figures{time_stamp}-paired")
os.makedirs(figure_path, exist_ok=True)
task_paired_list = Task_Paired_List(workspace_path, update_tasks=True)

In [6]:
# task_paired_list.plot_histogram(figure_path)

In [ ]:
class Task_Paired_Statistic():
    hostfile_pair_list:list[tuple] = [("hostfile1", "hostfile2"), ("hostfile3", "hostfile4"), ("hostfile5", "hostfile6")]
    nodes_pair_list:list[tuple] = [("pi1,pi2", "pi3,pi4"), ("pi1,pi3", "pi2,pi4"), ("pi1,pi4", "pi2,pi3")]
    headers = ['node_pair1', 'node_pair2', 'variables', 'task_type', 'perf_type', 'average', 'std', 'max', 'min', '1%', '5%', '1%_avg', '5%_avg']
    def __init__(self, task_paired_list:Task_Paired_List, update_data=True):
        self.data:pd.DataFrame = None
        if update_data or os.path.exists(f'data{time_stamp}-paired-statistic.csv') == False:
            self.nodes_perf_statistic(task_paired_list)
            self.data.sort_values(['perf_type', 'node_pair1', 'variables', 'task_type'], inplace=True)
            self.data.to_csv(f'data{time_stamp}-paired-statistic.csv', index=False)
        else:
            self.data = pd.read_csv(f'data{time_stamp}-paired-statistic.csv')

    def nodes_perf_statistic(self, task_paired_list:Task_Paired_List):
        '''
        
        '''
        data_row = []
        for node_pair in Task_Paired_Statistic.nodes_pair_list:
            data_node = task_paired_list.data[(task_paired_list.data['node_pair1'] == node_pair[0]) & (task_paired_list.data['node_pair2'] == node_pair[1])]
            variables_list = data_node['variables'].unique()
            for variables in variables_list:
                data_var = data_node[data_node['variables'] == variables]
                for res_type in ['time_ave_inter', 'time_ave_single', 'speed_ave_inter', 'speed_ave_single']:
                    task_type = 'interleaved' if 'inter' in res_type else 'single'
                    perf_type = 'total_time' if 'time' in res_type else 'MB/s'
                    average = data_var[res_type].mean()
                    std = data_var[res_type].std()
                    max = data_var[res_type].max()
                    min = data_var[res_type].min()
                    if 'time' in res_type:
                        one_percent = data_var[res_type].quantile(0.99)
                        five_percent = data_var[res_type].quantile(0.95)
                        average_one_percent = data_var[data_var[res_type] > one_percent][res_type].mean()
                        average_five_percent = data_var[data_var[res_type] > five_percent][res_type].mean()
                    else:
                        one_percent = data_var[res_type].quantile(0.01)
                        five_percent = data_var[res_type].quantile(0.05)
                        average_one_percent = data_var[data_var[res_type] < one_percent][res_type].mean()
                        average_five_percent = data_var[data_var[res_type] < five_percent][res_type].mean()
                    data_row.append([node_pair[0], node_pair[1], variables, task_type, perf_type, average, std, max, min, one_percent, five_percent, average_one_percent, average_five_percent])
                    
        if self.data is None:
            self.data = pd.DataFrame(data_row, columns=Task_Paired_Statistic.headers)
        else:
            self.data = pd.concat([self.data, pd.DataFrame(data_row, columns=Task_Paired_Statistic.headers)])


    

    def plot_statistic(self, save_path:str):
        '''
        绘制柱状图，每张图包含两对节点，柱状图的每个横轴是一个统计值：average,std,max,min,1%,5%,1%_avg,5%_avg，
        一个横轴坐标包含两个柱，其中一个柱是interleaved的，另一个柱是single的（归一化展示，single柱值为1）
        '''
        import matplotlib.pyplot as plt

        def plot_two_pairs_bar(ax, stats, data, perf_type, offset, color, task_type):
            values = []
            single_row = None
            # For interleaved, first get the corresponding single row for normalization.
            if task_type == 'interleaved':
                single_query = data[
                    (data['task_type'] == 'single') &
                    (data['perf_type'] == perf_type)
                ]
                if not single_query.empty:
                    single_row = single_query.iloc[0]
            for stat in stats:
                cur_query = data[
                    (data['task_type'] == task_type) &
                    (data['perf_type'] == perf_type)
                ]
                if cur_query.shape[0] > 0:
                    cur_val = cur_query[stat].iloc[0]
                    if task_type == 'single':
                        norm_val = 1
                    else:
                        if single_row is not None and single_row[stat] != 0:
                            norm_val = cur_val / single_row[stat]
                        else:
                            norm_val = 0
                else:
                    norm_val = 0
                values.append(norm_val)
            x = range(len(stats))
            ax.bar([p + offset for p in x], values, width=0.15, label=f"{task_type}", color=color)


        stats = ['average','std','max','min','1%','5%','1%_avg','5%_avg']
        for variable in self.data['variables'].unique():
            data_var = self.data[self.data['variables'] == variable]
            task_type_list = data_var['task_type'].unique()
            if len(task_type_list) != 2:
                continue
            for node_pair in Task_Paired_Statistic.nodes_pair_list:
                data_node_pair = data_var[(data_var['node_pair1'] == node_pair[0]) & (data_var['node_pair2'] == node_pair[1])]
                if data_node_pair.shape[0] == 0:
                    continue
                for perf_type in ['total_time', 'MB/s']:
                    fig, ax = plt.subplots(figsize=(8,4))
                    plot_two_pairs_bar(ax, stats, data_node_pair, perf_type, -0.03, 'orange', 'single')
                    plot_two_pairs_bar(ax, stats, data_node_pair, perf_type, 0.12, 'blue', 'interleaved')
                    ax.set_xticks(range(len(stats)))
                    # ax.set_yscale('log', base=10)
                    ax.set_xticklabels(stats, rotation=20)
                    ax.set_title(f"{node_pair} {variable} Bytes {perf_type}")
                    ax.legend()
                    file_name = f"Paired_Statistic-{variable}-{perf_type.replace('/', 'per')}-{node_pair}.png"
                    plt.tight_layout()
                    plt.savefig(os.path.join(save_path, file_name))
                    plt.close()

    def plot_statistic_nodes(self, save_path:str):
        '''
        绘制柱状图，每张图包含两对节点，柱状图的每个横轴是一个统计值：average,std,max,min,1%,5%,1%_avg,5%_avg，
        一个横轴坐标包含两个柱，其中一个柱是interleaved的，另一个柱是single的（归一化展示，single柱值为1）
        '''
        import matplotlib.pyplot as plt
        stats = ['average','std', '1%_avg','5%_avg']
        # For each statistic, create one figure that compares interleaved vs. single for a set of node pairs.
        for vars in self.data['variables'].unique():
            data_vars = self.data[self.data['variables'] == vars]
            for stat in stats:
                for perf_type in ['total_time', 'MB/s']:
                    data_pt = data_vars[data_vars['perf_type'] == perf_type]
                    title = f"Comparison of {stat} for {vars} Bytes in {perf_type}"
                    file_name = f"statistic-{vars}-{stat}-{perf_type.replace('/', 'per')}.png"
                    plot.plot_bar_with_discrete_x(data=data_pt, x='node_pair1', stat=stat, title=title, file_name=file_name, save_path=save_path)

                
tasks_paired_statistic = Task_Paired_Statistic(task_paired_list, update_data=True)
figure_path = os.path.join(workspace_path, f"figures{time_stamp}-paired-statistic")
os.makedirs(figure_path, exist_ok=True)
# tasks_paired_statistic.plot_statistic(figure_path)
tasks_paired_statistic.plot_statistic_nodes(figure_path)
